## Import libraries

In [ ]:
! pip install PyDictionary
! python -m spacy download en_core_web_md

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import re
import nltk
import spacy
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
import en_core_web_md
from nltk.corpus import wordnet
from PyDictionary import PyDictionary

## Load source datasets

In [ ]:
train_df = pd.read_csv("../input/commonlitreadabilityprize/train.csv")
train_df.drop(['url_legal','license','standard_error'], inplace=True, axis=1)
train_df.set_index("id", inplace=True)
print(f"train_df: {train_df.shape}\n")
train_df.head()

## Set percentage level 

**(number of words to be replaced for augmentation)**

In [ ]:
percent = 20

## Word Similarity Check

**(aim is to return the most similar word to the given input word)**

In [ ]:
def synalter_Noun_Verb(word, a1, POS):
    
    max_temp = -1
    flag = 0
    
    for i in a1:
        try:
            w1 = wordnet.synset(word + '.' + POS + '.01') 
            w2 = wordnet.synset(i + '.' + POS + '.01')

            if(max_temp < w1.wup_similarity(w2)):
                max_temp = w1.wup_similarity(w2)
                temp_name = i
                flag = 1
        except:
            f=0
            
    if flag == 0:
        max1 = -1.
        nlp = en_core_web_md.load()
        
        for i in a1:
            j=i.replace(' ', '')
            tokens = nlp(u'' + j)
            token_main = nlp(u'' + word)
            
            for token1 in token_main:
                if max1<float(token1.similarity(tokens)):
                    max1 = token1.similarity(tokens)
                    value = i
                    
        max1 = -1.
        return value
    
    else:
        return temp_name

## Text Replacement

1. Count the total number of Unique words and make a dictionary
2. Filter out unique words and eliminate 2 letters word & numbers
3. Make a list of Noun and Verb using spacy from the above-filtered list
4. Random selection from Noun and Verb along with above mention percentage
5. Loop all the randomly selected words
   * create a word_str variable to store a word.
   * Finding out all the synonyms using the thesaurus and making a list.
   * Checking If a selected word is Noun or Verb and than passing to `synalter_Noun_Verb(word, al, POS)`
   
   
6. Replacing the word with the new most similar word

In [ ]:
def text_replacement(text):

    synonyms = [] 
    antonyms = []

    output_text = text
    print(f"Original Text: \n\n{text}\n")
    print("="*90)

    words = text.split()
    counts = {}

    for word in words:
        if word not in counts:
            counts[word] = 0

        counts[word] += 1

    one_word = []

    for key, value in counts.items():
        if value == 1 and key.isalpha() and len(key)>2:
            one_word.append(key)

    noun = []
    verb = []

    nlp = en_core_web_md.load()
    doc = nlp(u''+' '.join(one_word))

    for token in doc:
        if  token.pos_ == 'VERB':
            verb.append(token.text)

        if  token.pos_ == 'NOUN':
            noun.append(token.text)

    all_main =verb + noun
    len_all = len(noun)+len(verb)
    final_value = int(len_all * percent /100)

    random.seed(0)
    dictionary=PyDictionary()
    temp = random.sample(range(0, len_all), final_value)

    for i in tqdm(temp):
        word_str = all_main[i]
        a1 = dictionary.synonym(word_str)
        
        if a1 is None:
            continue
        
        else:
            if i < len(verb):
                change_word=synalter_Noun_Verb(word_str, a1, 'v')
                search_word = re.search(r'\b('+word_str+r')\b', output_text)
                Loc = search_word.start()
                output_text = output_text[:int(Loc)] + '[' + change_word + ']' + output_text[int(Loc) + len(word_str):]

            else:
                change_word=synalter_Noun_Verb(word_str, a1, 'n')
                search_word = re.search(r'\b('+word_str+r')\b', output_text)
                Loc = search_word.start()
                output_text = output_text[:int(Loc)] + '[' + change_word + ']' + output_text[int(Loc) + len(word_str):]
    
    print("="*90)
    print(f'\nAugmented Text:\n\n{output_text}\n')

In [ ]:
%%time
text_replacement(train_df.iloc[0]['excerpt'])

In [ ]:
%%time
text_replacement(train_df.iloc[100]['excerpt'])

In [ ]:
%%time
text_replacement(train_df.iloc[1000]['excerpt'])